<a href="https://colab.research.google.com/github/andrelucianociotta/imersao_aula5/blob/main/aula5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -U -q google-generativeai


In [6]:
import numpy as np
import pandas as pd

import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get("SECRET_KEY")
#GOOGLE_API_KEY="chave"
gnai.configure(api_key=api_key)

In [8]:
for m in genai.list_models():
 if 'embedContent' in m.supported_generation_methods:
  print(m.name)


models/embedding-001
models/text-embedding-004


In [11]:
#exemplo de embedding

title = "A proxima geração"
sample_text = ("título: a proxima..."
"\n"
"Artigo completo"
"\n"
"Gemini API e google - ia generativa")

embeddings = gnai.embed_content(model="models/embedding-001",
                                content=sample_text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")

print(embeddings)


{'embedding': [0.033553667, -0.03932113, -0.0062470324, 0.004518621, 0.077964395, -0.011642371, -0.029627556, -0.04013623, 0.03186723, 0.028820809, -0.013619968, 0.004320812, -0.067676194, -0.014475476, 0.01784971, -0.022853384, 0.011015434, 3.2100656e-05, -0.041533172, 0.0010540467, -0.02682191, 0.02595023, -0.03144489, -0.052004185, 0.005153657, 0.007558664, 0.024172304, -0.037639312, -0.03955074, -0.004452491, -0.0591194, 0.036463283, -0.037751388, -0.018610539, -0.015919736, -0.072099045, -0.032329913, -0.046794605, -0.0030907306, 0.0016880521, 0.021436028, -0.070247315, 0.013279663, 0.021037664, 0.005402996, -0.046119437, 0.029466737, 0.027912386, 0.046989493, -0.07404465, 0.046922244, 0.023296615, 0.07039288, -0.038031783, -0.036830958, -0.010319425, 0.07015117, -0.04611048, 0.005004943, 0.002984554, -0.016376149, 0.023277197, 0.014430969, 0.04174935, 0.002235187, -0.034628924, -0.0233572, 0.004451822, 0.035116613, 0.0063114725, 0.0031734062, -0.02778279, 0.044810973, -0.04417273

In [23]:
DOCUMENT1 = {
    "title": "Operando o Sistema de Controle Climático",
    "content": "Seu Googlecar possui um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central. Temperatura: O botão de temperatura controla o temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir o fluxo de ar: O botão de fluxo de ar controla a quantidade de fluxo de ar dentro do carro. velocidade: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite selecionar o modo desejado. O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Frio: O carro soprará ar frio para dentro do carro. Calor: O carro soprará ar quente para dentro do carro. para descongelar."}
DOCUMENT2 = {
    "title": "Tela sensível ao toque",
    "content": "Seu Googlecar possui uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e controle de temperatura. Para usar a tela sensível ao toque, basta tocar no ícone desejado. Por exemplo, você pode tocar no ícone para obter rotas até seu destino ou toque no ícone \"Música\" para reproduzir suas músicas favoritas."}
DOCUMENT3 = {
    "title": "Mudando de marcha",
    "content": "Seu Googlecar possui transmissão automática. Para mudar de marcha, basta mover a alavanca de mudança para a posição desejada. Estacionar: Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Marcha-atrás: Esta posição A posição é usada para dar ré: Esta posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. . Baixo: Esta posição é usada para dirigir em neve ou outras condições escorregadias."}
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]



In [24]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operando o Sistema de Controle Climático,Seu Googlecar possui um sistema de controle cl...
1,Tela sensível ao toque,Seu Googlecar possui uma grande tela sensível ...
2,Mudando de marcha,Seu Googlecar possui transmissão automática. P...


In [36]:
model = "models/embedding-001"

In [37]:
def embed_fn(title, text):
  return gnai.embed_content(model=model,
                                content=text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [38]:
df["Embeddings"] = df.apply(lambda row:embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operando o Sistema de Controle Climático,Seu Googlecar possui um sistema de controle cl...,"[-0.019136094, -0.029979248, -0.030025043, 0.0..."
1,Tela sensível ao toque,Seu Googlecar possui uma grande tela sensível ...,"[0.01914887, -0.032459095, 0.015414452, 0.0195..."
2,Mudando de marcha,Seu Googlecar possui transmissão automática. P...,"[-0.0064704963, -0.026031032, -0.00623164, 0.0..."


In [40]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = gnai.embed_content(model=model,
                                content=consulta,
                                task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]



In [45]:
consulta = "Como faco para trocar marcha um carro do google?"
trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)




Seu Googlecar possui transmissão automática. Para mudar de marcha, basta mover a alavanca de mudança para a posição desejada. Estacionar: Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Marcha-atrás: Esta posição A posição é usada para dar ré: Esta posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. . Baixo: Esta posição é usada para dirigir em neve ou outras condições escorregadias.


reescrever a resposta buscando na IA do google

In [47]:
prompt = f"Reescreva esse texto de uma forma mais sussinta e descontraida: {trecho}"

model_2 = gnai.GenerativeModel("gemini-1.0-pro")
response = model_2.generate_content(prompt)
print(response.text)


**Sobre as marchas do seu Googlecar:**

* **Estacionar:** Quando você tá na lagoa, as rodas ficam presas no lugar.
* **Marcha-ré:** Usa pra dar ré, tipo quando você errou o caminho.
* **Neutro:** Serve pra ficar parado, como no sinal ou no trânsito.
* **Baixa:** Ótima pra neve ou quando tá escorregadio.
